In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
!pip install langgraph

Defaulting to user installation because normal site-packages is not writeable
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 26.9 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Attempting uninstall: xxhash
    Found existing installation: xxhash 3.4.1
    Uninstalling xxhash-3.4.1:
      Successfully uninstalled xxhash-3.4.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.3
    Uninstalling pydantic-2.7.3:
      Succes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.3 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.3.62 which is incompatible.
langchain 0.2.3 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.3.42 which is incompatible.
langchain-community 0.2.4 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.3.62 which is incompatible.
langchain-community 0.2.4 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.3.42 which is incompatible.
langchain-google-genai 1.0.6 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.3.62 which is incompatible.
langchain-text-splitters 0.2.1 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.3.62 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade

In [82]:
!python.exe -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [84]:
import openai
import os

from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda

In [5]:
openai.api_key = "OPEN_AI_API_KEY"

In [6]:
audio_file_path = "Record1.mp3" 

In [ ]:
def transcribe_audio(file_path):
    with open(file_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            response_format="text"  
        )
    return transcript

In [8]:
def save_transcript_to_file(transcript, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(transcript)
    print(f"Transcription saved to {output_path}")

if __name__ == "__main__":
    try:
        result = transcribe_audio(audio_file_path)
        save_transcript_to_file(result, "call_transcript.txt")
    except Exception as e:
        print(f"Error: {e}")

Transcription saved to call_transcript.txt


### **DAG Agnetic AI Workflow**

In [110]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
import openai
import os
from typing import Annotated, TypedDict

In [111]:
# === Define Schema using TypedDict ===
class State(TypedDict):
    audio_path: str
    transcript: str
    clean_text: str
    summary: str
    sentiment: str
    note: str
    evaluation: str

# === 1. STT Agent ===
def stt_agent(state) -> Annotated[dict, "transcript"]:
    audio_path = state["audio_path"]
    with open(audio_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            response_format="text"
        )
    return {"transcript": transcript}

In [112]:
# === 2. Preprocessing Agent ===
def preprocess_agent(state) -> Annotated[dict, "clean_text"]:
    transcript = state["transcript"]
    clean = transcript.replace("um", "").replace("uh", "").replace("you know", "")
    clean = " ".join(clean.split())
    return {"clean_text": clean}

In [113]:
# === 3. Combined Summary and Sentiment Agent ===
def summary_sentiment_agent(state) -> Annotated[dict, "summary | sentiment"]:
    clean_text = state["clean_text"]

    prompt_summary = f"Please summarize the following customer conversation:\n{clean_text}"
    summary = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_summary}]
    )["choices"][0]["message"]["content"]

    prompt_sentiment = f"Analyze the sentiment of the following conversation (positive/negative/neutral):\n{clean_text}"
    sentiment = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_sentiment}]
    )["choices"][0]["message"]["content"]

    return {"summary": summary, "sentiment": sentiment}

In [114]:
# === 4. Note Writer Agent ===
def note_writer_agent(state) -> Annotated[dict, "note"]:
    summary = state["summary"]
    sentiment = state["sentiment"]
    prompt = f"Summary: {summary}\nSentiment: {sentiment}\nPlease generate a customer service note based on the above."
    note = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )["choices"][0]["message"]["content"]
    return {"note": note}

In [115]:
# === 5. Evaluation Agent ===
def evaluation_agent(state) -> Annotated[dict, "evaluation"]:
    prompt = f"""Evaluate the accuracy of the following summary:
    Original Text:\n{state['clean_text']}
    Summary:\n{state['summary']}
    Rate it from 1 to 5 and explain your reasoning."""
    evaluation = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )["choices"][0]["message"]["content"]
    return {"evaluation": evaluation}

In [116]:
# === 6. Construct LangGraph DAG ===
builder = StateGraph(State)

builder.add_node("STT", RunnableLambda(stt_agent))
builder.add_node("Preprocess", RunnableLambda(preprocess_agent))
builder.add_node("SummarySentiment", RunnableLambda(summary_sentiment_agent))
builder.add_node("NoteWriter", RunnableLambda(note_writer_agent))
builder.add_node("Evaluation", RunnableLambda(evaluation_agent))

builder.set_entry_point("STT")
builder.add_edge("STT", "Preprocess")
builder.add_edge("Preprocess", "SummarySentiment")
builder.add_edge("SummarySentiment", "NoteWriter")
builder.add_edge("NoteWriter", "Evaluation")
builder.set_finish_point("Evaluation")


In [ ]:

# === 7. Execute Graph ===
graph = builder.compile()

initial_state = {"audio_path": "Short_Record1.mp3"}  
output = graph.invoke(initial_state)

# === 9. Output Results ===
print("\nFinal Note:\n", output["note"])
print("\nEvaluation:\n", output["evaluation"])


Final Note:
 Subject: Customer Service Training - Key Phrases for Professional Interactions

Dear Team,

As part of our ongoing commitment to excellent customer service, we are introducing a new training lesson that outlines 36 essential English phrases to enhance your interactions with customers. This training will be particularly beneficial for those of you working in call centers or handling customer calls regularly.

The lesson covers a variety of scenarios, including:

1. **Greeting Customers:** Learn how to introduce yourself and the company professionally to set a positive tone at the start of each call.

2. **Handling Complaints:** Gain key phrases that help you express empathy, apologize sincerely, and reassure customers that assistance is on the way.

3. **Transferring Calls:** Use polite and clear communication to inform customers when transfers are necessary, while ensuring they are comfortable with the process.

4. **Gathering Information:** Understand the importance of e

In [124]:
print(output['sentiment'])

The sentiment of the conversation is neutral. The text is an instructional piece, providing guidance on how to handle customer service calls. It does not express any personal emotions or opinions, but rather, it outlines professional customer service phrases and procedures for dealing with calls, including both routine answers and handling complaints. The tone is informative and professional, with the aim of educating, rather than conveying positive or negative feelings.


In [125]:
print(output['summary'])

In this customer service training lesson, 36 useful English phrases for professional customer interactions are introduced, particularly beneficial for those working in call centers or handling customer calls. The lesson covers various scenarios, including greeting customers, handling complaints, and transferring calls. Key phrases for answering calls professionally include introducing oneself and the company, and inquiring about how to assist the caller. When dealing with complaints, it's important to express empathy, apologize, and assure the customer of assistance. Additionally, if the caller needs to be transferred, polite phrases for doing so are provided, ensuring the customer is informed and agrees to the process. The lesson also emphasizes the importance of gathering necessary customer information when the service representative is equipped to handle the concern.


In [ ]:
print(output['evaluation'])

"I would rate this summary a 4 out of 5.\n\nHere's why:\n1. **Coverage**: The summary captures the main points of the original text effectively, mentioning the introduction of 36 useful phrases for customer service, focusing on call centers, and covering scenarios such as greeting customers, handling complaints, and transferring calls. \n\n2. **Clarity**: It clearly explains the purpose of the lesson, the scenarios covered, and the key phrases for professional interaction, making it easy for someone to understand what the lesson entails.\n\n3. **Detail**: While the summary conveys the primary elements of the original text, it simplifies and condenses some of the detailed examples provided, which is typical for a summary, but a few specific examples could add more depth to the understanding of certain phrases.\n\n4. **Accuracy**: The summary accurately reflects the content of the original text, but it omits the specific structure of the greeting examples and the empathy-related response